## Data Preparation 

In this notebook we write a stored procedure using **t-sql** to get a denormalised table so that we can use it for further tasks such as training ML models.

In [ ]:
-- stored procedure 
-- result: vendor performance attributes table
--

CREATE or ALTER PROCEDURE [dbo].[vendor_performance_report_sp]

(
    @materialcategorycode   varchar(15),
    @PeriodFrom             date,
    @PeriodUpto             date

)
AS
BEGIN
SET NOCOUNT ON

    DROP TABLE [coe].[dbo].[scm_vendor_performance]
    DROP TABLE [coe].[dbo].[scm_vendor_performance_final]
    -- DROP TABLE [coe].[dbo].[vendor_internal_performance_ratings]
    DROP TABLE [coe].[dbo].[_temp_mrn]
    DROP TABLE [coe].[dbo].[_temp_gin]
    DROP TABLE [coe].[dbo].[_temp_scm_po_no_count]
    DROP TABLE [coe].[dbo].[_temp_scm_offer_item_count]
    DROP TABLE [coe].[dbo].[_temp_scm_po_vendor_performance_price]

    CREATE TABLE [coe].[dbo].scm_vendor_performance
    (
        vp_Vendor_Code varchar(15),
        vp_PO_Number varchar(30),
        vp_Material_Code varchar(15),
        vp_sum_po_Quantity money,
        vp_po_pending_Quanity money,
        vp_po_Delivery_End_Date Datetime,
        vp_max_recived_end_date DateTime,
        vp_received_Quanity money,
        vp_accepted_Quantity money,
        vp_rejected_Quantity money ,
        vp_late_Accepted_Quantity money,
        vp_delivery_due int,
        vp_delivery_intime int,
        vp_accepted int,
        vp_Rejected int,
        vp_late_delivery int,
        vp_PO_Type_Code int,
        vp_offer_number varchar(30),
        vp_max_mrn_date datetime
    )

    -- insert query
    --

    INSERT INTO [coe].[dbo].scm_vendor_performance
        (
        vp_Vendor_Code,
        vp_PO_Number,
        vp_Material_Code,
        vp_sum_po_Quantity,
        vp_po_Delivery_End_Date,
        vp_po_pending_Quanity,
        vp_max_recived_end_date,
        vp_received_Quanity,
        vp_accepted_Quantity,
        vp_rejected_Quantity,
        vp_late_Accepted_Quantity,
        vp_delivery_due,
        vp_delivery_intime,
        vp_accepted,vp_Rejected,
        vp_late_delivery,
        vp_PO_Type_Code,
        vp_offer_number
        )

    -- po
    --
    select  hpo_ba_code, dpo_po_number, dpo_material_code,
    sum(dpo_qty - dpo_cancelled_qty),
    hpo_delivery_end_date,
    sum(dpo_pending_qty),
    NULL,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    hpo_po_type_detail_code,
    hpo_offer_number

    from    [coe].[eipscm].scm_h_purchase_orders_pq_hdfs inner join [coe].[eipscm].scm_d_purchase_orders_pq_hdfs on (hpo_po_number=dpo_po_number)
    inner join [coe].[eipmas].gen_m_jobs_pq_hdfs tmpjob on (hpo_job_code = tmpjob.mjob_job_code)
    inner join [coe].[eipmas].gen_m_materials_pq_hdfs on mmat_material_code = dpo_material_code
    inner join [coe].[eippmp].gen_l_material_category_material_group_pq_hdfs cml on cml.lmcmg_mg_code = mmat_mg_code and cml.lmcmg_company_code = 1 and cml.lmcmg_material_category_code in (@materialcategorycode)

    where hpo_po_date between @PeriodFrom and @PeriodUpto

    group by    hpo_ba_code, 
    dpo_po_number, 
    dpo_material_code, 
    hpo_delivery_end_date, 
    hpo_po_type_detail_code, 
    hpo_offer_number
        
    having sum(dpo_qty - dpo_cancelled_qty) > 0

    order by hpo_ba_code, 
    dpo_po_number, 
    dpo_material_code,
    hpo_delivery_end_date

    -- mrn
    --

    select hmrn_po_number,
    dmrn_material_code 
    , sum(dmrn_accepted_qty) as dmrn_accepted_qty,
    sum(dmrn_received_qty) as dmrn_received_qty,
    sum(dmrn_rejected_qty) as dmrn_rejected_qty,
    max(dmrn_dc_date) as dmrn_dc_date,
    max(hmrn_mrn_date) as hmrn_mrn_date
    into [coe].[dbo]._temp_mrn

    from [coe].[eipscm].scm_h_mrn_pq_hdfs, [coe].[eipscm].scm_d_mrn_pq_hdfs
    where hmrn_mrn_number = dmrn_mrn_number and hmrn_company_code = 1  and exists(select * from [coe].[dbo].scm_vendor_performance where vp_po_number = hmrn_po_number and vp_material_code = dmrn_material_code)
    group by hmrn_po_number, dmrn_material_code

    -- update-mrn-condition-1
    --

    update [coe].[dbo].scm_vendor_performance
    set vp_accepted_quantity = dmrn_accepted_qty,
    vp_received_quanity = p.dmrn_received_qty,                              
    vp_rejected_quantity= p.dmrn_rejected_qty,
    vp_max_recived_end_date = dmrn_dc_date,
    vp_max_mrn_date = hmrn_mrn_date 
    from [coe].[dbo]._temp_mrn p
    where p.hmrn_po_number = vp_po_number and
    p.dmrn_material_code=vp_material_code

    -- update-mrn-condition-2
    --

    update [coe].[dbo].scm_vendor_performance
    set vp_late_accepted_quantity = dmrn_accepted_qty
    from [coe].[dbo]._temp_mrn p 
    where p.hmrn_po_number = vp_po_number
    and p.dmrn_material_code = vp_material_code
    and vp_po_delivery_end_date < dmrn_dc_date
    and vp_po_delivery_end_date < vp_max_recived_end_date

    -- gin
    --

    select  hgin.hgin_po_number,  dgin.dgin_material_code, sum(dgin.dgin_inspection_pending_qty) as gin_pending_qty
    into [coe].[dbo]._temp_gin
    from [coe].[eipscm].scm_h_gin_pq_hdfs hgin, [coe].[eipscm].scm_d_gin_pq_hdfs dgin
    where   hgin.hgin_gin_number = dgin.dgin_gin_number and dgin.dgin_inspection_pending_qty >0 and hgin.hgin_company_code= 1 and exists (select top 1 'x' from [coe].[dbo].scm_vendor_performance where hgin.hgin_po_number = vp_po_number and dgin.dgin_material_code = vp_material_code)
    group by hgin.hgin_po_number , dgin.dgin_material_code

    --update gin
    --

    update [coe].[dbo].scm_vendor_performance 
    set  vp_po_pending_quanity = vp_po_pending_quanity+gin_pending_qty
    from [coe].[dbo]._temp_gin
    where hgin_po_number = vp_po_number and dgin_material_code = vp_material_code and gin_pending_qty>0

    --update-gin
    --

    update  [coe].[dbo].scm_vendor_performance 
    set   vp_po_pending_quanity= vp_po_pending_quanity + tmpvendor3.pending_qty
    from [coe].[dbo].scm_vendor_performance tmpvendor1 ,

    (
    select hboe.hboe_po_number , 
    dboe.dboes_material_code,
    sum (dboe.dboes_pending_qty) as pending_qty

    from    [coe].[eipscm].scm_h_bill_of_entry_pq_hdfs as hboe, 
    [coe].[eipscm].scm_d_bill_of_entry_summary_pq_hdfs as dboe,
    [coe].[dbo].scm_vendor_performance tmpvendor2

    where   hboe.hboe_boe_number = dboe.dboes_boe_number and
    tmpvendor2.vp_po_number = hboe.hboe_po_number and
    tmpvendor2.vp_material_code = dboe.dboes_material_code and
    dboe.dboes_pending_qty >0
    
    group by hboe.hboe_po_number , dboe.dboes_material_code 
    
    ) tmpvendor3 

    where   tmpvendor3.hboe_po_number = tmpvendor1.vp_po_number and tmpvendor3.dboes_material_code = tmpvendor1.vp_material_code and tmpvendor3.pending_qty>0

    -- update-values
    --

    update  [coe].[dbo].scm_vendor_performance
    set  vp_delivery_due  = 1 
    where vp_po_pending_quanity > 0


    update  [coe].[dbo].scm_vendor_performance
    set vp_late_delivery  = 1 
    where vp_late_accepted_quantity >0 and vp_po_pending_quanity <=0


    update  [coe].[dbo].scm_vendor_performance
    set vp_delivery_intime = 1 
    where vp_po_pending_quanity <=0 and vp_late_delivery = 0

    update  [coe].[dbo].scm_vendor_performance
    set vp_accepted = 1 
    where vp_accepted_quantity >0 and vp_po_pending_quanity <=0

    update  [coe].[dbo].scm_vendor_performance
    set vp_rejected = 1 
    where   vp_rejected_quantity >0

    -- write table 
    -- 

    create  table [coe].[dbo].scm_vendor_performance_final
    (      
        vpfl_vendor_code           varchar(15),
        vpfl_po_number_count       int,
        vpfl_total_po_value        money,
        vpfl_po_item_count         int,
        vpfl_delivery_due          int,
        vpfl_delivery_intime       int,
        vpfl_acceptedfull          int,
        vpfl_late_delivery         int,
        vpfl_delivery_weightage    money,
        vpfl_quality_weightage     money,
        vpfl_accepted_rate         money,
        vpfl_price_competitiveness money,
        vpfl_offer_items_count     int,
        vpfl_offer_po_items_count  int,
        vpfl_offer_material_count  int 
    )

    insert into [coe].[dbo].scm_vendor_performance_final
    ( 
                    
        vpfl_vendor_code,
        vpfl_po_number_count,
        vpfl_total_po_value,
        vpfl_po_item_count,
        vpfl_delivery_due,
        vpfl_delivery_intime,
        vpfl_acceptedfull,
        vpfl_late_delivery,
        vpfl_delivery_weightage,
        vpfl_quality_weightage,
        vpfl_accepted_rate,
        vpfl_price_competitiveness,
        vpfl_offer_material_count 
                
    )                            

    select  vp_vendor_code, 
    0, 
    0,
    count(1),
    sum(vp_delivery_due), 
    sum(vp_delivery_intime),
    sum(vp_accepted), 
    sum(vp_late_delivery), 
    0,
    0,
    0,
    0,
    0 
    from [coe].[dbo].scm_vendor_performance
    group by vp_vendor_code 

    --update1

    update [coe].[dbo].scm_vendor_performance_final
    set  vpfl_delivery_weightage = ((cast(tmpvendorfinal.vpfl_delivery_intime as money)/(cast(tmpvendorfinal.vpfl_po_item_count as money) - cast(tmpvendorfinal.vpfl_delivery_due as money)))*40)
    from   [coe].[dbo].scm_vendor_performance_final tmpvendorfinal
    where  (tmpvendorfinal.vpfl_po_item_count - tmpvendorfinal.vpfl_delivery_due)>0 

    --update2

    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_quality_weightage = ((cast(tmpvendorfinal.vpfl_acceptedfull as money)/(cast(tmpvendorfinal.vpfl_po_item_count as money) - cast(tmpvendorfinal.vpfl_delivery_due as money)))*50)
    from   [coe].[dbo].scm_vendor_performance_final tmpvendorfinal
    where  (tmpvendorfinal.vpfl_po_item_count - tmpvendorfinal.vpfl_delivery_due)>0 


    -- count number of POs
    -- 

    CREATE TABLE [coe].[dbo]._temp_scm_po_no_count 
    ( 
        vp_Po_No_Count      int,
        vp_vendor_Code      varchar(15),
        vp_Total_Po_Value   money
    )

    Insert Into [coe].[dbo]._temp_scm_po_no_count
    (
        vp_Po_No_Count, vp_vendor_Code, 
        vp_Total_Po_Value
    )    

    select count(hpo_po_number) as po_no_count, hpo_ba_code as vendor_code, sum(hpo_po_net_value)     
    from [coe].[eipscm].scm_h_purchase_orders_pq_hdfs inner join [coe].[eipmas].gen_m_jobs_pq_hdfs tmpjob on (hpo_job_code= tmpjob.mjob_job_code)    
    where hpo_po_date between @PeriodFrom and @PeriodUpto and exists (select top 1 'x' from [coe].[dbo].scm_vendor_performance vp where vp.vp_po_number=hpo_po_number)       
    group by hpo_ba_code


    -- offer item count
    -- 

    CREATE TABLE [coe].[dbo]._temp_scm_offer_item_count 
    ( 
        vp_Offer_item_Count int,
        vp_vendor_Code varchar(15)
    )

    Insert Into [coe].[dbo]._temp_scm_offer_item_count
    ( 
        
        vp_Offer_item_Count, 
        vp_vendor_Code 

    )                              

    select count(distinct doffr_material_code) as offer_item_count, hpo_ba_code as vendor_code
    from [coe].[eipscm].scm_h_purchase_orders_pq_hdfs inner join [coe].[eipmas].gen_m_jobs_pq_hdfs tmpjob on (hpo_job_code= tmpjob.mjob_job_code) 
    inner join [coe].[eipscm].scm_h_offer_pq_hdfs on (hoffr_offer_number = hpo_offer_number) inner join [coe].[eipscm].scm_d_offer_pq_hdfs on (doffr_offer_number = hoffr_offer_number)
    where hpo_po_date between @PeriodFrom and @PeriodUpto and exists (select top 1 'x' from [coe].[dbo].scm_vendor_performance vp where vp.vp_po_number = hpo_po_number)
    group by hpo_ba_code

    --update 
    -- 

    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_po_number_count = tmppo.vp_po_no_count
    from [coe].[dbo].scm_vendor_performance_final tmpvendorfinal, [coe].[dbo]._temp_scm_po_no_count tmppo
    where tmpvendorfinal.vpfl_vendor_code = tmppo.vp_vendor_code 

    --update                                 
    ---upadte values to #scm_vendor_performance_final for document wise  offer item count
                            
    update tmpvendorfinal
    set    tmpvendorfinal.vpfl_offer_material_count = tmpoffer.vp_offer_item_count
    from   [coe].[dbo].scm_vendor_performance_final tmpvendorfinal, [coe].[dbo]._temp_scm_offer_item_count tmpoffer
    where  tmpvendorfinal.vpfl_vendor_code = tmpoffer.vp_vendor_code                                                          
                                    
    --update3                    
    ---upadte values to #scm_vendor_performance_final for total po value in lacks
                            
    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_total_po_value = (tmpvendor.vp_total_po_value / 100000)
    from   [coe].[dbo].scm_vendor_performance_final tmpvendorfinal,[coe].[dbo]._temp_scm_po_no_count tmpvendor
    where  tmpvendorfinal.vpfl_vendor_code = tmpvendor.vp_vendor_code

    -- price 
    -- 

    Create table [coe].[dbo]._temp_scm_po_vendor_performance_price 
    (      
        vp_Vendor_Code        varchar(15), 
        vp_PO_Number          varchar(30),
        vp_po_items_count     int,
        vp_offer_number       varchar(30), 
        vp_offer_items_Count  int 
    )

    Insert into [coe].[dbo]._temp_scm_po_vendor_performance_price 
    (
        vp_Vendor_Code,vp_PO_Number, 
        vp_po_items_count,vp_offer_number,vp_offer_items_Count
    )                                     

    Select  vp_Vendor_Code,
    vp_PO_Number,
    COUNT(1) poitemscount,
    vp_offer_number,
    0
    from [coe].[dbo].scm_vendor_performance
    where vp_PO_Type_Code not in (25,26)

    group by    vp_Vendor_Code, vp_PO_Number, vp_offer_number

    --update1

    update [coe].[dbo]._temp_scm_po_vendor_performance_price  
    set vp_offer_items_count = tmpoffers.offer_items_count 
    from [coe].[dbo]._temp_scm_po_vendor_performance_price tmpprice,    
    (
    select doffr_offer_number, 
    count(distinct doffr_material_code
    ) offer_items_count 

    from [coe].[dbo]._temp_scm_po_vendor_performance_price tmpvendor, [coe].[eipscm].scm_d_offer_pq_hdfs
    inner join [coe].[eipmas].gen_m_materials_pq_hdfs on mmat_material_code = doffr_material_code and mmat_company_code = 1     
    inner join [coe].[eippmp].gen_l_material_category_material_group_pq_hdfs cml on cml.lmcmg_mg_code = mmat_mg_code and cml.lmcmg_company_code = 1 and cml.lmcmg_material_category_code = @materialcategorycode
    where doffr_offer_number = tmpvendor.vp_offer_number                                                                       
    group by doffr_offer_number) tmpoffers
    where tmpprice.vp_offer_number=tmpoffers.doffr_offer_number


    --udpate2

    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_offer_items_count = offeritemcount, 
        vpfl_offer_po_items_count=poitemcount
    from [coe].[dbo].scm_vendor_performance_final tmpfinal2 ,  
    (
    select vp_vendor_code,
    sum(vp_po_items_count) poitemcount, 
    sum(vp_offer_items_count) offeritemcount
    from [coe].[dbo]._temp_scm_po_vendor_performance_price tmpprice, [coe].[dbo].scm_vendor_performance_final tmpfinal1
    where tmpprice.vp_vendor_code=tmpfinal1.vpfl_vendor_code
    group by vp_vendor_code
    ) tmpprice2
    
    where tmpfinal2.vpfl_vendor_code=tmpprice2.vp_vendor_code

    --update3

    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_price_competitiveness = (cast(vpfl_offer_po_items_count as money)/cast(vpfl_offer_items_count as money))*10.00
    where vpfl_offer_items_count > 0
    and vpfl_offer_po_items_count<=vpfl_offer_items_count
                
    --update4
                            
    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_price_competitiveness = 10,
    vpfl_offer_po_items_count = 0,
    vpfl_offer_items_count = 0
    where vpfl_offer_po_items_count is null and
    vpfl_offer_items_count is null

    --update5

    update [coe].[dbo].scm_vendor_performance_final
    set vpfl_accepted_rate = vpfl_delivery_weightage + vpfl_price_competitiveness 
    from [coe].[dbo].scm_vendor_performance_final

    -- final table 
    -- 

    create table [coe].[dbo].vendor_internal_performance_ratings 
    (  

        [material_category_code]        varchar(15),
        [vendor_code]                   varchar(15),
        [period_from]                   date,
        [period_upto]                   date,
        [po_number_count]               int,
        [total_po_value]                money,
        [po_item_count]                 int,
        [delivery_due]                  int,
        [delivery_intime]               int, 
        [acceptedfull]                  int,
        [late_delivery]                 int,
        [delivery_weightage]            money,
        [quality_weightage]             money,
        [accepted_rate]                 money,
        [price_competitiveness]         money,
        [offer_material_count]          int,
        [extracted_on]                  datetime

    )

    insert into [coe].[dbo].vendor_internal_performance_ratings
    (
        material_category_code, 
        vendor_code, 
        period_from, 
        period_upto, 
        po_number_count, 
        total_po_value,
        po_item_count, 
        delivery_due, 
        delivery_intime, 
        acceptedfull, 
        late_delivery, 
        delivery_weightage, 
        quality_weightage, 
        accepted_rate, 
        price_competitiveness, 
        offer_material_count, 
        extracted_on
    )

    select @materialcategorycode material_category_code, 
    vpfl_vendor_code, 
    @PeriodFrom, 
    @PeriodUpto,
    vpfl_po_number_count,
    vpfl_total_po_value,
    vpfl_po_item_count,
    vpfl_delivery_due,
    vpfl_delivery_intime,
    vpfl_acceptedfull,
    vpfl_late_delivery,
    vpfl_delivery_weightage,
    vpfl_quality_weightage,
    vpfl_accepted_rate,
    vpfl_price_competitiveness,
    vpfl_offer_material_count, 
    getdate()
    from [coe].[dbo].scm_vendor_performance_final

    -- drop temporary tables 
    --

    DROP TABLE [coe].[dbo].scm_vendor_performance
    DROP TABLE [coe].[dbo].scm_vendor_performance_final
    DROP TABLE [coe].[dbo].[_temp_mrn]
    DROP TABLE [coe].[dbo].[_temp_gin]
    DROP TABLE [coe].[dbo].[_temp_scm_po_no_count]
    DROP TABLE [coe].[dbo].[_temp_scm_offer_item_count]
    DROP TABLE [coe].[dbo].[_temp_scm_po_vendor_performance_price]

END

In [ ]:
-- run sp with parameters
--
-- ('M001', 'M002', 'M003', 'M004', 'M005', 'M006', 'M007', 'M008', 'M009', 'M010', 'M011' ,'M012', 'M013', 'M014', 'M015')
use coe 
go

EXEC vendor_performance_report_sp @materialcategorycode = 'm015', @periodFrom = '2019-01-01', @PeriodUpto = '2020-07-30'

In [ ]:
    -- create table [coe].[dbo].vpr_m001 
    -- (  

    --     [material_category_code]        varchar(15),
    --     [vendor_code]                   varchar(15),
    --     [period_from]                   date,
    --     [period_upto]                   date,
    --     [po_number_count]               int,
    --     [total_po_value]                money,
    --     [po_item_count]                 int,
    --     [delivery_due]                  int,
    --     [delivery_intime]               int, 
    --     [acceptedfull]                  int,
    --     [late_delivery]                 int,
    --     [delivery_weightage]            money,
    --     [quality_weightage]             money,
    --     [accepted_rate]                 money,
    --     [price_competitiveness]         money,
    --     [offer_material_count]          int,
    --     [extracted_on]                  datetime

    -- )

insert into [coe].[dbo].vpr_m001
    (
        material_category_code, 
        vendor_code, 
        period_from, 
        period_upto, 
        po_number_count, 
        total_po_value,
        po_item_count, 
        delivery_due, 
        delivery_intime, 
        acceptedfull, 
        late_delivery, 
        delivery_weightage, 
        quality_weightage, 
        accepted_rate, 
        price_competitiveness, 
        offer_material_count, 
        extracted_on
    )

    select material_category_code, 
    vendor_code, 
    period_from, 
    period_upto, 
    po_number_count,
    total_po_value,
    po_item_count,
    delivery_due, 
    delivery_intime, 
    acceptedfull, 
    late_delivery, 
    delivery_weightage, 
    quality_weightage, 
    accepted_rate, 
    price_competitiveness, 
    offer_material_count, 
    extracted_on
    from [coe].[dbo].vendor_internal_performance_ratings


In [ ]:
-- drop exisiting stored proc

if exists(select * from dbo.sysobjects where id = object_id(N'[dbo].[vendor_performance_report_sp]'))
drop procedure [dbo].[vendor_performance_report_sp]

In [ ]:
--select * from [coe].[dbo].vpr_m001

In [ ]:
-- create column
--

alter table [COE].[dbo].[vpr_m001] add [vendor_name] nvarchar(max) 



In [ ]:
-- update table with vendor names
-- 

update a 
set a.vendor_name = b.Vendor_Name 
from [COE].[dbo].[vpr_m001] a left join [COE].[sqlmas].[gen_m_vendors_pq_hdfs] b
on a.vendor_code = b.Vendor_Code 
